## Agile Hardware Design
***
# Inheritance

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Inheritance in Scala
* Inheritance with Chisel
* Type parameterization

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Why Use (Object-Oriented) Inheritance?

* Increase productivity through _reuse_
  * Less effort to reuse a component than implementing from scratch

* _Inheritance_ is most helpful for reuse between similar components
  * What about different components that behave the same in same cases?
  * What about different components that present the same interface?

* Due to diversity of potential reuse cases, Scala has a diversity of inheritance mechanisms

## Scala `class` Inheritance

* Simply use the `extend` keyword
  * Can only extend one class at a time

* Can overide fields with `override`

In [ ]:
class Parent(name: String) {
    val phrase = "hello"
    
    def greet() { println(s"$phrase $name") }
}

val p = new Parent("Kate")
p.greet

class Child(name: String) extends Parent(name) {
    override val phrase = "hola"
}

val c = new Child("Pablo")
c.greet

## Scala `abstract class` Inheritance

* Sometimes don't want to provide implementations of the inherited things
* Can't instantiate an abstract class, must inherit from it
* Full multiple inheritance, will need to consider _trait_

In [ ]:
abstract class Parent(name: String) {
    val phrase: String
    
    def greet() { println(s"$phrase $name") }
}

class InEnglish(name: String) extends Parent(name) {
    val phrase = "hello"
}

val e = new InEnglish("Kate")
e.greet

class InSpanish(name: String) extends Parent(name) {
    val phrase = "hola"
}

val s = new InSpanish("Pablo")
s.greet

## Example Type Hierarchy: Scala's Immutable Collections

<img src="images/collections-immutable-diagram.svg" alt="Scala's immutable collections type hierarchy" style="width:60%;margin-left:auto;margin-right:auto"/>

[source](https://docs.scala-lang.org/overviews/collections-2.13/overview.html)

## Types of Reuse in Using Chisel

* To enable agile development, we have been going after reuse in different places:

* _**Parameterized hardware generators**_ - are hopefully sufficiently flexible to be used in more places
  * Both at module level as well as functionality not even wrapped in a Module

* _**Composable/customizable Bundles**_ - can reduce effort defining interfaces

* _**via Inheritance**_ - (today) similar modules can share functionality
  * Design decision of using different classes (inheritance) or more generator parameters
  * _Note:_ Chisel itself is implemented using inheritance (e.g. `extend Module`)


## Chisel Simple `abstract class` Inheritance

In [ ]:
abstract class UnaryOperatorModule(width: Int) extends Module {
    def op(x: UInt): UInt
    val io = IO(new Bundle {
        val in = Input(UInt(width.W))
        val out = Output(UInt(width.W))
    })
    io.out := op(io.in)
}

class PassThruMod(width: Int) extends UnaryOperatorModule(width) {
    def op(x: UInt) = x
}

class NegMod(width: Int) extends UnaryOperatorModule(width) {
    def op(x: UInt) = ~x
}

println(getVerilog(new PassThruMod(8)))

## Chisel Example - Building Operator Library (1/2)

In [ ]:
//  c = op(a,b)
abstract class DecoupledOperator(width: Int) extends Module {
    val io = IO(new Bundle {
        val a = Flipped(Decoupled(UInt(width.W)))
        val b = Flipped(Decoupled(UInt(width.W)))
        val c = Decoupled(UInt(width.W))
    })

    def op(a: UInt, b: UInt): UInt

    val buffer = Reg(UInt(width.W))
    val full = RegInit(false.B)
    io.a.ready := !full
    io.b.ready := !full
    io.c.valid := full
    io.c.bits := buffer
    when (io.a.fire && io.b.fire && !full) {
        buffer := op(io.a.bits, io.b.bits)
        full := true.B
    }
    when (io.c.fire) {
        full := false.B
    }
}

## Chisel Example - Building Operator Library (2/2)

In [ ]:
class DecoupledAdd(width: Int) extends DecoupledOperator(width) {
    def op(a: UInt, b: UInt): UInt = a + b
}

class DecoupledSub(width: Int) extends DecoupledOperator(width) {
    def op(a: UInt, b: UInt): UInt = a - b
}

object DecoupledFactory {
    def apply(op: String, width: Int): DecoupledOperator = op match {
        case "+" => new DecoupledAdd(width)
        case "-" => new DecoupledSub(width)
        case _ => throw new Exception(s"Couldn't find $op")
    }
}

println(getVerilog(DecoupledFactory("+", 8)))

// test(DecoupledFactory("+", 8)) { c =>
//     for (cycle <- 0 until 5) {
//         c.io.a.bits.poke((cycle+1).U)
//         c.io.a.valid.poke(true.B)
//         c.io.b.bits.poke(cycle.U)
//         c.io.b.valid.poke(true.B)
//         c.io.c.ready.poke(true.B)
//         println(s"$cycle:$cycle ${c.io.c.bits.peek} ${c.io.c.valid.peek}")
//         c.clock.step()
//     }
// }


## Scala `trait`

* More flexible than `abstract class` in most ways
  * Can inherit from multiple traits
  * Can't take constructor parameters
* Sometimes refered to as _mixin_
  * Good conceptual model: think of inheriting from trait to "mix in" some needed functionality (or interface)
* Great in Chisel for adding a little functionality to different types of modules

## Example Chisel Use of `trait`

* Want to add standardized way of querying module
* Using MultiIOModule allows for not defining `io` all at once

In [ ]:
trait PrintInSim extends MultiIOModule {
    val printEnable = IO(Input(Bool()))
    
    def msg: String

    when (printEnable) {
        printf(p"$msg\n")
    }
}

class CounterMod extends MultiIOModule with PrintInSim {
    val out = IO(Output(UInt(8.W)))
    def msg = "hello from counter"
    val count = Counter(255)
    out := count.value
}

test(new CounterMod) { c =>
    c.printEnable.poke(false.B)
    c.clock.step(2)
    c.printEnable.poke(true.B)
    c.clock.step(2)
}

## Templating Types

* Possible to parameterize a type (e.g. a generic) in Scala
* Typically want to use _type bounds_ to ensure functionality is there
* Can sometimes come run into issue of [_type erasure_](https://squidarth.com/scala/types/2019/01/11/type-erasure-scala.html)

## Chisel with Templated Type

* With Chisel, usually want to specify _type bounds_ to ensure is hardware
* Need to explicitly pass _gen_ to get type constructor

In [ ]:
class GenericPassThru[T <: chisel3.Data](gen: T) extends Module {
    val io = IO(new Bundle {
        val in = Input(gen)
        val out = Output(gen)
    })
    io.out := io.in
}

println(getVerilog(new GenericPassThru(UInt(8.W))))

## Chisel Type Hierarchy

<img src="images/chisel_hierarchy.svg" alt="Chisel's type hierarchy" style="width:60%;margin-left:auto;margin-right:auto"/>

[source](https://github.com/chipsalliance/chisel3)

## Templating Our Queue (from last lecture)

In [ ]:
class MyQueueV7[T <: chisel3.Data](numEntries: Int, gen: T, pipe: Boolean=true) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(gen))
        val deq = Decoupled(gen)
    })
    require(numEntries > 1)
//     require(isPow2(numEntries))
    val entries = Mem(numEntries, gen)
    val enqIndex = Counter(numEntries)
    val deqIndex = Counter(numEntries)
    val maybeFull = RegInit(false.B)
    val indicesEqual = enqIndex.value === deqIndex.value
    val empty = indicesEqual && !maybeFull
    val full = indicesEqual && maybeFull
    if (pipe)
        io.enq.ready := !full || io.deq.ready
    else
        io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex.value)
    when (io.deq.fire =/= io.enq.fire) {
        maybeFull := io.enq.fire
    }
    when (io.deq.fire) {
        deqIndex.inc()
    }
    when (io.enq.fire) {
        entries(enqIndex.value) := io.enq.bits
        enqIndex.inc()
    }
}

In [ ]:
class QueueModel(numEntries: Int) {
    val mq = scala.collection.mutable.Queue[Int]()
    var deqReady = false

    def attemptEnq(elem: Int) { 
        if (enqReady()) mq += elem
    }

    // call first within a cycle
    // improve with Option & None
    def attemptDeq() = if (deqReady) mq.dequeue() else -1
    
    def enqReady() = mq.size < numEntries || (mq.size == numEntries && deqReady)
    def deqValid() = mq.nonEmpty
}

In [ ]:
def simCycle(qm: QueueModel, c: MyQueueV7[UInt], enqReady: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    if (deqReady && qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqReady.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqReady)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueueV7(3, UInt(8.W))) { c =>
    val qm = new QueueModel(3)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, true, false, 2)
    simCycle(qm, c, true, false, 3)
    simCycle(qm, c, false, true)
}

## Project Overview

### GOAL: gain experience developing/revising a generator

### Main Details
* Working in pairs (or individually)
* Pick an idea to build a generator for (contact instructor if need suggestion)
* Will _propose/design/develop/test/optimize/revise/document/present_ generator
* Ballpark for size/complexity: ~1.5x most recent homework problems
  * Building largely from scratch, so much more to do

## Project Timeline

* Week 6 (next week) - find a partner and brainstorm ideas
* Week 7 - propose project & get instructor feedback
* Week 8 - close the loop early and keep developing
* Week 9 - complete initial development & start revising
* Week 10 - finalize/polish project & present

## Project Deliverables

* 2/14 (by class time) - initial proposal (<1 page)
  * What will generator do and what interface/parameters will it have?
  * Consider how to bootstrap, test, and what features that can be deferred
* 2/28 - Link to working repo (can be feature incomplete)
  * Close the loop early, and build from there
* 3/9 or 3/11 - Presentation
* 3/14 - Links to final repo & revised presentation
  * Following presentation, will have time to make small revisions
  * Encouraged (but not required) to post publicly